# EfficientNet backbone

In [6]:
import numpy as np

In [7]:
from DatasetLoader import DatasetLoader

In [8]:
datasetloader = DatasetLoader(dataset_root='../dataset/', img_resize=(224,224))

In [9]:
filenames, images = datasetloader.loadTest()

100%|██████████| 12186/12186 [02:29<00:00, 81.67it/s]


In [10]:
# fit pyTorch input 
features = np.transpose(images, (0,3,1,2))
features.shape

(12186, 3, 224, 224)

# Model Prediction

In [37]:
import torch
from torch import nn

from efficientnet_pytorch import EfficientNet

'''
reference:
https://github.com/lukemelas/EfficientNet-PyTorch
'''

class predictor_EfficientNet(nn.Module):
    def __init__(self, img_size = (224,224), n_classes=42):
        super().__init__()
        
        self.n_classes = n_classes
        self.img_size = img_size
        
        # load/freeze backbone
        self.backbone = None
        self._loadBackbone()
        self._freezeBackbone()
        
        # predictor
        ## b0
        self._avg_pooling = nn.AdaptiveAvgPool2d(1)
        self._dropout = nn.Dropout(0.2)
        self._fc = nn.Linear(1280, self.n_classes)
        
        ## b3
#         self._avg_pooling = nn.AdaptiveAvgPool2d(1)
#         self._dropout = nn.Dropout(0.3)
#         self._fc = nn.Linear(1536, self.n_classes)
    
        ## b7
#         self._avg_pooling = nn.AdaptiveAvgPool2d(1)
#         self._dropout = nn.Dropout(0.5)
#         self._fc = nn.Linear(2560, self.n_classes)
        
        
    def forward(self, x):
        bs = x.shape[0] # batch size
        
        # feature extractor
        x = self.backbone.extract_features(x)
        
        # predictor
        x = self._avg_pooling(x)
        x = x.view(bs, -1)
        x = self._dropout(x)
        x = self._fc(x)
        
        return x
    
    def _loadBackbone(self):
        self.backbone = EfficientNet.from_pretrained('efficientnet-b0')
        
    def _freezeBackbone(self):
        for param in self.backbone.parameters():
            param.requires_grad = False
        
model = predictor_EfficientNet()

Loaded pretrained weights for efficientnet-b0


In [38]:
model.load_state_dict(torch.load('./model/EfficientNet/b0_v1.pkl'))

In [39]:
import torch
from torch import nn
from torch.autograd import Variable

# epoch

# check GPU
gpu_mode = torch.cuda.is_available()
if gpu_mode:
    model.cuda()
    
model.eval()

interval = 100
res = []
res_p = []
for i in range(0, len(features), interval):
    x = torch.FloatTensor(features[i:i+interval])
    if gpu_mode:
        x = x.cuda()
    pred = model(x)
    _, pred_ = torch.max(pred, 1)
    
    res_p += pred.tolist() # shape : (b, 42)
    res += pred_.tolist() # shape : (b,)
    print('idx : %s done' % i)

idx : 0 done
idx : 100 done
idx : 200 done
idx : 300 done
idx : 400 done
idx : 500 done
idx : 600 done
idx : 700 done
idx : 800 done
idx : 900 done
idx : 1000 done
idx : 1100 done
idx : 1200 done
idx : 1300 done
idx : 1400 done
idx : 1500 done
idx : 1600 done
idx : 1700 done
idx : 1800 done
idx : 1900 done
idx : 2000 done
idx : 2100 done
idx : 2200 done
idx : 2300 done
idx : 2400 done
idx : 2500 done
idx : 2600 done
idx : 2700 done
idx : 2800 done
idx : 2900 done
idx : 3000 done
idx : 3100 done
idx : 3200 done
idx : 3300 done
idx : 3400 done
idx : 3500 done
idx : 3600 done
idx : 3700 done
idx : 3800 done
idx : 3900 done
idx : 4000 done
idx : 4100 done
idx : 4200 done
idx : 4300 done
idx : 4400 done
idx : 4500 done
idx : 4600 done
idx : 4700 done
idx : 4800 done
idx : 4900 done
idx : 5000 done
idx : 5100 done
idx : 5200 done
idx : 5300 done
idx : 5400 done
idx : 5500 done
idx : 5600 done
idx : 5700 done
idx : 5800 done
idx : 5900 done
idx : 6000 done
idx : 6100 done
idx : 6200 done
idx 

# Submission

In [29]:
import pandas as pd
import os

### predict actual category for submission

In [28]:
categories = [str(e) if e >= 10 else '0'+str(e) for e in res]
categories[:10]

['20', '27', '04', '22', '39', '20', '04', '39', '13', '33']

In [32]:
# to df
df = pd.DataFrame({'filename': filenames, 'category': categories})

output_path = './submission/EfficientNet/b3_v1.csv'
if os.path.exists(output_path):
    raise FileExistsError('[ERROR] Submission file already exist, please change output path')
else:
    df.to_csv(output_path, index=False)

FileExistsError: [ERROR] Submission file already exist, please change output path

### predict probability for ensemble method

In [40]:
df = pd.DataFrame({'filename': filenames})

# categories col
res_p = np.array(res_p)
for category in range(res_p.shape[1]): # 42
    df[category] = res_p[:,category]
    
df.head()

,filename,0,1,2,3,4,5,6,7,8,...,32,33,34,35,36,37,38,39,40,41
0,fd663cf2b6e1d7b02938c6aaae0a32d2.jpg,16.695902,-36.201328,58.980919,1.614652,51.591236,13.293336,-17.852489,34.784733,15.962917,...,21.361414,5.712780,36.532379,-19.003777,-6.449189,-17.838243,18.672947,-32.386887,13.563140,75.315491
1,c7fd77508a8c355eaab0d4e10efd6b15.jpg,-53.075459,-78.847702,-41.815739,-55.163521,-58.689976,4.952919,-72.957359,-50.608402,3.467971,...,16.982698,-33.724979,16.903425,-50.193691,81.529091,-54.419518,84.170631,8.955645,16.562439,23.564457
2,127f3e6d6e3491b2459812353f33a913.jpg,-36.251041,-46.770401,-33.250061,-3.830712,31.129425,-73.197792,-204.648819,-71.948158,-84.096298,...,-34.667522,-79.671089,-81.297340,-119.642227,-30.010406,-109.860809,-73.154709,-125.878555,-60.693352,-12.416172
3,5ca4f2da11eda083064e6c36f37eeb81.jpg,-82.446228,-107.006142,-10.706336,-11.712032,-31.671200,-59.663666,-62.395592,-44.479958,-59.401901,...,35.872925,3.736557,-16.101139,-8.906132,44.259792,3.295516,-15.485658,1.668562,-25.264904,-26.167925
4,46d681a542f2c71be017eef6aae23313.jpg,17.497965,14.791681,32.389725,-6.474938,17.631119,-38.943657,-7.876534,-3.170712,29.897381,...,33.100891,-3.814386,-49.428333,4.456917,15.624740,12.667063,14.100931,-17.583696,-6.945380,8.763266


In [47]:
output_path = './prediction/EfficientNet/b0_v1.csv'
if os.path.exists(output_path):
    raise FileExistsError('[ERROR] Submission file already exist, please change output path')
else:
    df.to_csv(output_path, index=False)

FileExistsError: [ERROR] Submission file already exist, please change output path